# condaインストール(conda install)
condacolab インストール後にセッションがクラッシュするが、インストールは成功しているため、以降の処理を実行する。<br>
Session crashes after installing condacolab, but the installation was successful, so proceed further.

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


# 必要パッケージインストール(install)

In [2]:
%%bash
conda create -n DAMO-YOLO python=3.7 -y
eval "$(conda shell.bash hook)" 
conda activate DAMO-YOLO

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/DAMO-YOLO

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    python-3.7.12              |hf930737_100_cpython        57.3 MB  conda-forge
    sqlite-3.40.0              |       h4ff8645_0         801 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        58.2 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  ca-certificates    conda-forge/linux-64::ca-certificates-2022.12.7-ha878542_0 None
  ld_impl_linux-64   conda-forge/linux-64::ld_impl



==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 22.11.1

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [3]:
!git clone https://github.com/tinyvision/damo-yolo.git
%cd damo-yolo/

!conda install pytorch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0 cudatoolkit=10.2 -c pytorch
!pip install -r requirements.txt
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install onnx==1.8.1
!pip install onnxruntime==1.8.0
!pip install onnx-simplifier==0.3.5
!pip install protobuf==3.19.0
!export PYTHONPATH=$PWD:$PYTHONPATH

Cloning into 'damo-yolo'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 191 (delta 54), reused 164 (delta 31), pack-reused 0
Receiving objects: 100% (191/191), 1.57 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/damo-yolo
Solving environment: | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 40.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 40.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.1/605.1 kB 11.5 MB/s eta 0:00:00
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.5-py3-none-any.whl size=12863 sha256=a0f9f0947cb1378b57531781a1681f3f5259c5462189405e1ff5dac0ebf8f476
  Stored in directory: /root/.cache/pip/wheels/fd/30/8e/518f4064830c3014a870bbc6abd8f6e5203804c96b07e83e29
Successfully built onnx-simplifier
Looking in indexes: https://py

# データセットダウンロード(Download Dataset)

In [4]:
%cd /content/

!git clone https://github.com/Kazuhito00/DAMO-YOLO-Colaboratory-Training-Sample

/content
Cloning into 'DAMO-YOLO-Colaboratory-Training-Sample'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 141 (delta 15), reused 10 (delta 2), pack-reused 110
Receiving objects: 100% (141/141), 43.26 MiB | 36.70 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [5]:
import os

# 独自のデータを使用する場合は、パスを指定してください
# Please fill in the path if you want to use your own data
dataset_directory = 'DAMO-YOLO-Colaboratory-Training-Sample/sample_dataset'

# 学習/検証データパス(train/validation data path)
train_directory = './train'
validation_directory = './validation'

# 学習データ格納ディレクトリ作成(Create training data storage directory)
os.makedirs(train_directory, exist_ok=True)
# 検証データ格納ディレクトリ作成(Create verification data storage directory)
os.makedirs(validation_directory, exist_ok=True)

In [6]:
import glob
import shutil
import random

# 学習データの割合(Percentage of training data)
train_ratio = 0.8

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(dataset_directory + '/*.xml'))
image_list = sorted(glob.glob(dataset_directory + '/*.jpg'))

file_num = len(annotation_list)

# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_directory)
        shutil.copy2(image_list[index], train_directory)
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], validation_directory)
        shutil.copy2(image_list[index], validation_directory)

# Pascal VOC形式 を MS COCO形式へ変換(Convert Pascal VOC format to MS COCO format)

In [7]:
!git clone https://github.com/Kazuhito00/convert_voc_to_coco.git

Cloning into 'convert_voc_to_coco'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 27 (delta 12), reused 14 (delta 6), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [8]:
!python convert_voc_to_coco/convert_voc_to_coco.py \
    train train/train.json \
    --start_image_id=0 \
    --reserve_category_num=80
!python convert_voc_to_coco/convert_voc_to_coco.py \
    validation validation/val.json \
    --start_image_id=10000000 \
    --reserve_category_num=80

Number of xml files: 40
Convert XML to JSON: 100% 40/40 [00:00<00:00, 8890.48it/s]
{'Fish': 221}
Success: train/train.json
Number of xml files: 9
Convert XML to JSON: 100% 9/9 [00:00<00:00, 7733.81it/s]
{'Fish': 48}
Success: validation/val.json


# 学習データディレクトリ準備(Training data directory preparation)

In [9]:
!mkdir custom_coco
!mkdir custom_coco/train
!mkdir custom_coco/val
!mkdir custom_coco/annotations

!cp -rf train/*.jpg custom_coco/train
!cp -rf validation/*.jpg custom_coco/val
!cp -rf train/train.json custom_coco/annotations
!cp -rf validation/val.json custom_coco/annotations

In [10]:
%cd /content/damo-yolo/damo/config

/content/damo-yolo/damo/config


In [11]:
!rm paths_catalog.py

In [12]:
%%writefile paths_catalog.py

# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
# Copyright (C) Alibaba Group Holding Limited. All rights reserved.
"""Centralized catalog of paths."""
import os


class DatasetCatalog(object):
    DATA_DIR = 'datasets'
    DATASETS = {
        'custom_coco_train': {
            'img_dir':
            '/content/custom_coco/train',
            'ann_file':
            '/content/custom_coco/annotations/train.json'
        },
        'custom_coco_val': {
            'img_dir':
            '/content/custom_coco/val',
            'ann_file':
            '/content/custom_coco/annotations/val.json'
        },
    }

    @staticmethod
    def get(name):
        if 'coco' in name:
            data_dir = DatasetCatalog.DATA_DIR
            attrs = DatasetCatalog.DATASETS[name]
            args = dict(
                root=os.path.join(data_dir, attrs['img_dir']),
                ann_file=os.path.join(data_dir, attrs['ann_file']),
            )
            return dict(
                factory='COCODataset',
                args=args,
            )
        elif 'voc' in name:
            data_dir = DatasetCatalog.DATA_DIR
            attrs = DatasetCatalog.DATASETS[name]
            args = dict(
                data_dir=os.path.join(data_dir, attrs['data_dir']),
                split=attrs['split'],
            )
            return dict(
                factory='PascalVOCDataset',
                args=args,
            )
        elif 'objects365' in name:
            data_dir = DatasetCatalog.DATA_DIR
            attrs = DatasetCatalog.DATASETS[name]
            args = dict(
                root=os.path.join(data_dir, attrs['img_dir']),
                ann_file=os.path.join(data_dir, attrs['ann_file']),
            )
            return dict(
                factory='Objects365',
                args=args,
            )
        return None

Writing paths_catalog.py


# DAMO-YOLO学習用コンフィグファイル生成

In [13]:
%cd /content/damo-yolo

/content/damo-yolo


In [14]:
!wget https://idstcv.oss-cn-zhangjiakou.aliyuncs.com/DAMO-YOLO/clean_models/damoyolo_tinynasL20_T.pth

--2022-12-11 07:35:45--  https://idstcv.oss-cn-zhangjiakou.aliyuncs.com/DAMO-YOLO/clean_models/damoyolo_tinynasL20_T.pth
Resolving idstcv.oss-cn-zhangjiakou.aliyuncs.com (idstcv.oss-cn-zhangjiakou.aliyuncs.com)... 47.92.17.241
Connecting to idstcv.oss-cn-zhangjiakou.aliyuncs.com (idstcv.oss-cn-zhangjiakou.aliyuncs.com)|47.92.17.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35846473 (34M) [application/octet-stream]
Saving to: ‘damoyolo_tinynasL20_T.pth’

damoyolo_tinynasL20 100%[===================>]  34.19M  4.97MB/s    in 16s     

2022-12-11 07:36:02 (2.10 MB/s) - ‘damoyolo_tinynasL20_T.pth’ saved [35846473/35846473]



In [15]:
%%writefile damoyolo_tinynasL20_T.py

#!/usr/bin/env python3
import os

from damo.config import Config as MyConfig

class Config(MyConfig):

    def __init__(self):
        super(Config, self).__init__()

        self.miscs.exp_name = os.path.split(
            os.path.realpath(__file__))[1].split('.')[0]
        self.miscs.eval_interval_epochs = 10
        self.miscs.ckpt_interval_epochs = 10

        # optimizer
        self.train.batch_size = 8
        self.train.total_epochs = 100
        self.train.base_lr_per_img = 0.01 / 64
        self.train.min_lr_ratio = 0.05
        self.train.weight_decay = 5e-4
        self.train.momentum = 0.9
        self.train.warmup_epochs = 5
        self.train.no_aug_epochs = 16

        self.test.batch_size = 8

        # augment
        self.train.augment.transform.image_max_range = (640, 640)
        self.train.augment.mosaic_mixup.mixup_prob = 0.15
        self.train.augment.mosaic_mixup.degrees = 10.0
        self.train.augment.mosaic_mixup.translate = 0.2
        self.train.augment.mosaic_mixup.shear = 0.2
        self.train.augment.mosaic_mixup.mosaic_scale = (0.1, 2.0)

        # finetuning file
        self.train.finetune_path = '/content/damo-yolo/damoyolo_tinynasL20_T.pth'
        
        # dataset path
        self.dataset.train_ann = ('custom_coco_train', )
        self.dataset.val_ann = ('custom_coco_val', )

        # backbone
        structure = self.read_structure(
            './damo/base_models/backbones/nas_backbones/tinynas_L20_k1kx.txt')
        TinyNAS = {
            'name': 'TinyNAS_res',
            'net_structure_str': structure,
            'out_indices': (2, 4, 5),
            'with_spp': True,
            'use_focus': True,
            'act': 'relu',
            'reparam': True,
        }

        self.model.backbone = TinyNAS

        GiraffeNeckV2 = {
            'name': 'GiraffeNeckV2',
            'depth': 1.0,
            'hidden_ratio': 1.0,
            'in_channels': [96, 192, 384],
            'out_channels': [64, 128, 256],
            'act': 'relu',
            'spp': False,
            'block_name': 'BasicBlock_3x3_Reverse',
        }

        self.model.neck = GiraffeNeckV2

        ZeroHead = {
            'name': 'ZeroHead',
            'num_classes': 80,
            'in_channels': [64, 128, 256],
            'stacked_convs': 0,
            'reg_max': 16,
            'act': 'silu',
            'nms_conf_thre': 0.05,
            'nms_iou_thre': 0.7
        }
        self.model.head = ZeroHead

Writing damoyolo_tinynasL20_T.py


# 暫定修正
参考：https://github.com/tinyvision/DAMO-YOLO/issues/24

In [16]:
with open('/content/damo-yolo/damo/dataset/datasets/mosaic_wrapper.py') as fp:
    file_lines = fp.read()

file_lines = file_lines.replace(
    'for i, s in enumerate(segments):',
    'for i, s in enumerate(segments):\n        if s.shape[0] == 0:\n            continue',
)

file_lines = file_lines.replace(
    '    if (n and segments is None) or (len(segments) != len(targets)):',
    '    segments = []\n    if (n and segments is None) or (len(segments) != len(targets)):',
)

with open('/content/damo-yolo/damo/dataset/datasets/mosaic_wrapper.py', mode="w") as fp:
    fp.write(file_lines)

# トレーニング

In [17]:
%cd /content/damo-yolo

/content/damo-yolo


In [18]:
!cp tools/train.py ./

In [19]:
!python -m torch.distributed.launch \
    --nproc_per_node=1 --node_rank=0 \
    train.py \
    -f damoyolo_tinynasL20_T.py

2022-12-11 07:36:07 | INFO     | damo.apis.detector_trainer:111 - args info: Namespace(config_file='damoyolo_tinynasL20_T.py', local_rank=0, opts=[], tea_ckpt=None, tea_config=None)
2022-12-11 07:36:07 | INFO     | damo.apis.detector_trainer:112 - cfg value:
╒═════════╤══════════════════════════════════════════════════════════════════════════════════╕
│ keys    │ values                                                                           │
╞═════════╪══════════════════════════════════════════════════════════════════════════════════╡
│ model   │ {'backbone': {'act': 'relu',                                                     │
│         │               'name': 'TinyNAS_res',                                             │
│         │               'net_structure_str': "[ {'class': 'ConvKXBNRELU', 'in': 3, 'k': "  │
│         │                                    "3, 'nbitsA': 8, 'nbitsW': 8, 'out': 24, "    │
│         │                                    "'s': 1},\n"                 

# ONNXエクスポート

In [20]:
%cd /content/damo-yolo

/content/damo-yolo


In [21]:
!cp tools/converter.py ./

In [22]:
!python converter.py \
    -f damoyolo_tinynasL20_T.py \
    -c workdirs/damoyolo_tinynasL20_T/epoch_100_ckpt.pth \
    --batch_size 1 \
    --img_size 640

2022-12-11 07:41:45.717 | INFO     | __main__:main:124 - args value: Namespace(batch_size=1, ckpt='workdirs/damoyolo_tinynasL20_T/epoch_100_ckpt.pth', config_file='damoyolo_tinynasL20_T.py', device='0', end2end=False, half=False, img_size=640, input='images', opset=11, ort=False, output='output', trt=False, trt_eval=False, trt_version=7)
2022-12-11 07:41:48.900 | INFO     | __main__:main:151 - loading checkpoint done.
2022-12-11 07:41:55.309 | INFO     | __main__:main:160 - backbone's params(M): 2.17, flops(G): 7.97, latency(ms): 5.578
neck's params(M): 5.71, flops(G): 8.24, latency(ms): 6.677
head's params(M): 0.60, flops(G): 1.92, latency(ms): 2.035
total latency(ms): 11.725, total flops(G): 18.14

/usr/local/lib/python3.8/site-packages/torch/tensor.py:590: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  war

# ダウンロード

In [23]:
from google.colab import files
files.download('damoyolo_tinynasL20_T.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>